In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
from bs4 import BeautifulSoup
import pandas as pd

/var/folders/4h/6_x41ljs40b6qtydhdtdtx280000gn/T/ipykernel_4927/1585860332.py:7: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
# downloading and saving the HTML content using Selenium
def download_html(driver, file_path):
    time.sleep(5)  # Wait for JavaScript to render
    # save the page source (HTML) to a file
    html_content = driver.page_source
    with open(file_path, 'w', encoding='utf-8') as file:
        file.write(html_content)

In [3]:
# extracting reviews
def get_reviews(soup):
    reviews_list = []

    # finding the review elements
    reviews = soup.find_all('div', {'data-hook': 'review'})
    if not reviews:
        print("No reviews found on this page.")
        exit()
        return reviews_list

    for review in reviews:
        try:
            # review title
            title_element = review.find('a', {'data-hook': 'review-title'})
            title = title_element.text.strip().split("\n")[-1] if title_element else 'No title'

            # review body
            body_element = review.find('span', {'data-hook': 'review-body'})
            body = body_element.text.strip() if body_element else 'No body'

            # review rating
            rating_element = review.find('i', {'data-hook': 'review-star-rating'})
            rating = rating_element.text.strip() if rating_element else 'No rating'

            reviews_list.append({
                "title": title,
                "body": body,
                "rating": rating,
            })
        except AttributeError as e:
            print(f"Error parsing review: {e}")
    
    return reviews_list

In [4]:
# main function, putting it into a data frame
def mein(base_url):
    html_file_path = 'amazon_reviews.html'
    options = webdriver.ChromeOptions()
    options.add_argument('--headless')  # 
    driver = webdriver.Chrome(options=options)

    dataframe = pd.DataFrame(columns = ["title", "body", "rating"])
    t = []
    b = []
    r = []

    try:
        for page_number in range(1,11):
            url = base_url + str(page_number)
            driver.get(url)
            download_html(driver, html_file_path)
            with open(html_file_path, 'r', encoding='utf-8') as file:
                html_content = file.read()
            soup = BeautifulSoup(html_content, 'html.parser')
            reviews = get_reviews(soup)
            if not reviews and page_number == 1:  # if no reviews found on the first page => break the loop
                break
            for review in reviews:
                t.append(review["title"])
                b.append(review["body"])
                r.append(review["rating"])

    finally:
        driver.quit()
    
    dataframe["title"] = t
    dataframe["body"] = b
    dataframe["rating"] = r
    print(dataframe)
    


In [5]:

if __name__ == '__main__':
    mein(base_url = 'https://www.amazon.com/Razer-BlackShark-V2-Gaming-Headset/product-reviews/B09PZG4R17/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_reviews')
    

                                                title  \
0   For the price, there aren’t many as good as this.   
1                                      Great headset!   
2   Not the most comfortable, but pretty good all ...   
3                                       price quality   
4   Designed and manufactured very well. They work...   
..                                                ...   
95                                             gaming   
96                                              great   
97  Immersive Gaming Experience: Gaming Headphones...   
98        The bass drivers really need to be cranked!   
99          Hard to beat these at this price point !!   

                                                 body              rating  
0   I never was a huge Razer fan but these are inc...  5.0 out of 5 stars  
1   I was looking for something comfortable with b...  5.0 out of 5 stars  
2   Bought this to replace a Plantronics Rig wired...  4.0 out of 5 stars  
3   The hea